In [1]:
import matplotlib.pyplot as plt
#%matplotlib inline
from PIL import Image # Install Pillow -> conda install anaconda::pillow or pip install pillow
import os
from skimage.io import  imread, imshow # Install scikit-image -> conda install scikit-image or pip install scikit-image
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.metrics import confusion_matrix, balanced_accuracy_score,  mean_squared_error, r2_score, f1_score
from sklearn.model_selection import KFold, train_test_split
import numpy as np

#import torch
import torch.nn as nn
import torch.optim as optim
from torch.functional import F
from torch.nn import Dropout

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Check if the code is running on Google Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    train_dataset_path = '/content/drive/MyDrive/Nicoli/data-students/TRAIN'
    test_dataset_path = '/content/drive/MyDrive/Nicoli/data-students/TEST'
else:
    # Load data from local file
    train_dataset_path = 'data-students/TRAIN'
    test_dataset_path = 'data-students/TEST'

# Now you can use file_path to load your data
#print("File path:", file_path)

#FIXED VARIABLES
IMG_WIDTH = 75
IMG_HEIGHT = 75
BATCH_SIZE = 32

#testing variables
seed = 42

In [3]:
#!pip install rembg
from rembg import remove
import io

def remove_background_pil(image):
    # Convert PIL image to bytes
    with io.BytesIO() as output_buffer:
        image.save(output_buffer, format='PNG')
        image_bytes = output_buffer.getvalue()

    # Use rembg to remove the background
    result = remove(image_bytes)

    # Convert the result binary data back to a PIL image
    result_image = Image.open(io.BytesIO(result))

    # Fill transparent pixels with black
    result_image = fill_transparent_pixels_with_black(result_image)

    # Convert the image to RGB mode if it's not already
    if result_image.mode != 'RGB':
        result_image = result_image.convert('RGB')

    return result_image

def fill_transparent_pixels_with_black(image):
    # Convert image to RGBA mode if it's not already
    if image.mode != 'RGBA':
        image = image.convert('RGBA')

    # Get the image data as a pixel access object
    pixel_data = image.load()

    # Iterate over each pixel
    width, height = image.size
    for x in range(width):
        for y in range(height):
            # Check if the pixel is transparent
            if pixel_data[x, y][3] == 0:
                # Set the pixel color to black (RGB: 0, 0, 0, Alpha: 255)
                pixel_data[x, y] = (0, 0, 0, 255)

    return image

In [4]:
from torchvision.transforms import v2

# Define your custom transformation function
augmentation = transforms.Compose([v2.RandomAffine(degrees=(-20, 20), translate=(0.1, 0.2), scale=(0.5, 0.9))])

def removeBackground(image):
    # Apply your custom background removal function
    processed_image = remove_background_pil(image) # remove_background(removal_model, image)

    # Apply other transformations if needed
    transform = transforms.Compose([
        transforms.Resize((IMG_WIDTH, IMG_HEIGHT)),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    return transform(processed_image)

def Augment(image):
    # Apply your custom background removal function
    transform = transforms.Compose([
        augmentation,
        transforms.Resize((IMG_WIDTH, IMG_HEIGHT)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    return transform(image)

def input_transform(image):
    # Apply your custom background removal function
    transform = transforms.Compose([
        augmentation,
        transforms.Resize((IMG_WIDTH, IMG_HEIGHT)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    return transform(image)

normal_transform = transforms.Compose([
        transforms.Resize((IMG_WIDTH, IMG_HEIGHT)),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])


In [5]:
#COMBINE DATASET WITH RANDOMLY AUGMENTED DATASETS

torch.cuda.is_available()
True
from torchvision.transforms import v2
import shutil
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#background_removed_path = '/content/drive/MyDrive/Nicoli/data-students/TRAIN-NOBG'

background_removed_path = 'data-students/TRAIN-NOBG'

'''
# Assuming you already have your original dataset and transformed dataset
original_dataset = datasets.ImageFolder(root=train_dataset_path, transform=normal_transform)

# Create a DataLoader
dataloader = DataLoader(original_dataset, batch_size=None, shuffle=False)

# Iterate through the DataLoader
for image_path, target in original_dataset.imgs:
    class_name = original_dataset.classes[target]
    class_path = os.path.join(background_removed_path, class_name)
    os.makedirs(class_path, exist_ok=True)

    # Load the image
    image = Image.open(image_path)

    # Apply the transformation
    transformed_image = removeBackground(image)

    # Save the transformed image in the corresponding class folder
    image_filename = os.path.basename(image_path)
    image_save_path = os.path.join(class_path, image_filename)
    transformed_image_pil = transforms.ToPILImage()(transformed_image)
    transformed_image_pil.save(image_save_path)  # Save the transformed image

'''


'\n# Assuming you already have your original dataset and transformed dataset\noriginal_dataset = datasets.ImageFolder(root=train_dataset_path, transform=normal_transform)\n\n# Create a DataLoader\ndataloader = DataLoader(original_dataset, batch_size=None, shuffle=False)\n\n# Iterate through the DataLoader\nfor image_path, target in original_dataset.imgs:\n    class_name = original_dataset.classes[target]\n    class_path = os.path.join(background_removed_path, class_name)\n    os.makedirs(class_path, exist_ok=True)\n\n    # Load the image\n    image = Image.open(image_path)\n\n    # Apply the transformation\n    transformed_image = removeBackground(image)\n\n    # Save the transformed image in the corresponding class folder\n    image_filename = os.path.basename(image_path)\n    image_save_path = os.path.join(class_path, image_filename)\n    transformed_image_pil = transforms.ToPILImage()(transformed_image)\n    transformed_image_pil.save(image_save_path)  # Save the transformed image\n

In [6]:
from torch.utils.data import ConcatDataset
nobg_dataset = datasets.ImageFolder(root=background_removed_path, transform=normal_transform)


# Define how many times you want to enlarge the dataset
enlarge_factor = 1

# Create a list to hold the datasets
combined_datasets = [nobg_dataset]

# Add the transformed dataset to the list multiple times
for _ in range(enlarge_factor):
    transformed_dataset = datasets.ImageFolder(root=background_removed_path, transform=Augment)
    combined_datasets.append(transformed_dataset)

# Concatenate the datasets into a single dataset
enlarged_dataset = ConcatDataset(combined_datasets)


test_set_size = 0.4
#get train & test for K-MEANS
train_val_dataset, test_dataset = train_test_split(enlarged_dataset, test_size = test_set_size, random_state=seed) #random_state = randomizer seed
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
len(enlarged_dataset)

1036

In [8]:
#to print the label (AUXILIAR)
label_str = [
    "12 - Don't Go Left or Right",
    "13 - Don't Go Right",
    "24 - Go Right",
    "37 - Children crossing",
    "38 - Dangerous curve to the right",
    "39 - Dangerous curve to the left",
    "44 - Go left or straight",
    "50 - Fences",
    "6 - Speed limit (70km/h)"
]
label_str_id = [
    "12",
    "13",
    "24",
    "37",
    "38",
    "39",
    "44",
    "50",
    "6"
]

In [9]:
def voting(trained_models,data_loader,type, modelName):
    #type = 1 -> data loader with no labels (test folder)
    #type = 0 -> data loader with labels

    iter = len(trained_models)
    #save the models
    for i, model in enumerate(trained_models):
        filename = f"model_fold_" + str(i) + "_" + modelName + ".pth"
        torch.save(model, filename)


    if type == 0: #data loader with labels
        #pred matrix
        predictions = []
        for i in range(iter):
            filename = f"model_fold_" + str(i) + "_" + modelName + ".pth"
            torch.save(model, filename)
            model.eval()
            #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)
            i = 0
            with torch.no_grad():
                    model_predictions = []

                    for inputs, labels in data_loader:
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        _, predicted = outputs.max(1)
                        model_predictions.append(predicted)
                    predictions_aux = torch.cat(model_predictions, dim=0).cpu()
                    predictions.append(predictions_aux)

        predictions_matrix = np.vstack(predictions)

        #voting
        num_classes = predictions_matrix.max() + 1

        class_votes = np.zeros((num_classes, predictions_matrix.shape[1])) #(num_classes, num_predictions)

        for col in range(predictions_matrix.shape[1]):
            unique_classes, class_counts = np.unique(predictions_matrix[:, col], return_counts=True)
            class_votes[unique_classes, col] = class_counts

        most_voted_classes = np.argmax(class_votes, axis=0) #pred

        #labels
        full_dataset = []
        for batch in data_loader:
            inputs, labels = batch
            full_dataset.append((inputs, labels))
            y = torch.cat([labels for _, labels in full_dataset], dim=0) #labels

        # evaluation
        # Compute confusion matrix and F1 score

        conf_mat = confusion_matrix(y, most_voted_classes)
        f1 = f1_score(y, most_voted_classes, average='weighted')
        bal_acc = balanced_accuracy_score(y, most_voted_classes)
        #precision = precision_score(y, most_voted_classes, average='weighted')
        #recall = recall_score(y, most_voted_classes, average='weighted')

        print('Confusion Matrix:\n', conf_mat)
        print('F1 Score: ', f1)
        print('B_acc: ', bal_acc)
        #print('Precision: ', precision)
        #print('Recall: ', recall)

        return y, most_voted_classes

    else: #from test folder - no labels
        predictions = []
        for i in range(iter):
            model = torch.load(f"model_fold_{i}.pth")
            model.eval()
            #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)
            i = 0
            with torch.no_grad():
                    model_predictions = []

                    for i, inputs in enumerate(data_loader):
                        inputs = inputs.to(device)
                        outputs = model(inputs)
                        predicted = torch.argmax(outputs, dim=1)
                        model_predictions.append(predicted)
                    predictions_aux = torch.cat(model_predictions, dim=0).cpu()
                    predictions.append(predictions_aux)

        predictions_matrix = np.vstack(predictions)

        #voting
        num_classes = predictions_matrix.max() + 1

        class_votes = np.zeros((num_classes, predictions_matrix.shape[1])) #(num_classes, num_predictions)

        for col in range(predictions_matrix.shape[1]):
            unique_classes, class_counts = np.unique(predictions_matrix[:, col], return_counts=True)
            class_votes[unique_classes, col] = class_counts

        most_voted_classes = np.argmax(class_votes, axis=0) #pred

        return most_voted_classes

In [10]:
 # PREDICT TEST FOLDER AND CREATE CSV FILE ----------------------------------------------------

def createCSV(model, test_dataset_loader,type, name):
    import csv

    data = []

    #directory where you want to save the CSV file
    try:
        import google.colab
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False

    if IN_COLAB:
        save_dir = '/content/drive/MyDrive/Nicoli'
    else:
        #save_dir = r"C:\Users\Nicoli Leal\Desktop\MEEC\2 semestre\Aprendizagem Computacional Avançada\Project-1"
        save_dir = "/home/stefanotrenti/AML/project"


    #file name
    csv_file = os.path.join(save_dir, name)

    if type == "voting":
      most_voted_classes = voting(model, test_dataset_loader,1)
      for i in range(len(most_voted_classes)):
        predicted_class = int(most_voted_classes[i])  # Extract the integer value
        data.append({"ID": i+1, "Class": label_str_id[predicted_class]}) #, "Name": label_str[test_predictions]})

    else:
      for i, images in enumerate(test_dataset_loader):

          images = images.to(device)
          # Forward pass
          outputs = model(images)
          test_predictions = torch.argmax(outputs, dim=1)

          images = images.cpu().numpy()
          predicted_classes = test_predictions.cpu().numpy()

          # Iterate over the batch
          predicted_class = int(predicted_classes[0])  # Extract the integer value
          data.append({"ID": i+1, "Class": label_str_id[predicted_class]}) #, "Name": label_str[test_predictions]})


    # Define the field names
    fields = ["ID", "Class"]#, "Name"]

    # Write data to CSV file
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fields)

        # Write the header
        writer.writeheader()

        # Write the data rows
        for row in data:
            writer.writerow(row)

    print("CSV file created successfully.")
    return

In [11]:
# DEFINE EVALUATE FUNCTION
def evaluate_network(model,dataset_loader, to_device=True):
    # X given input data
    # y corresponding target labels
    full_dataset = []
    for batch in dataset_loader:
        # Assuming each batch is a tuple (inputs, labels)
        inputs, labels = batch
        full_dataset.append((inputs, labels))

        # Concatenate all data points into a single tensor
        X = torch.cat([inputs for inputs, _ in full_dataset], dim=0)
        y = torch.cat([labels for _, labels in full_dataset], dim=0)


    # Set the model to evaluation mode
    model.eval()
    if to_device:
      # Assuming you're using GPU (if available)
        #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        X = X.to(device)
        y = y.to(device)
        #model = model.to(device)

    # Run the model on the test data
    with torch.no_grad():
        outputs = model(X)
        _, predicted = torch.max(outputs.data, 1)

    # Convert tensors to numpy arrays
    if to_device:
        predicted = predicted.to("cpu")

    predicted_np = predicted.cpu().numpy()
    test_target_np = y.cpu().numpy()


    # Compute confusion matrix and F1 score
    conf_mat = confusion_matrix(test_target_np, predicted_np)
    f1 = f1_score(test_target_np, predicted_np, average='weighted')
    bal_acc = balanced_accuracy_score(y.cpu(), predicted_np)

    #print('Confusion Matrix:\n', conf_mat)
    print('F1 Score: ', f1)
    print('B_acc: ', bal_acc)

    return conf_mat, f1, bal_acc

In [12]:
import os,sys

def saveResults(model, directory, name, modelName, optim, loss):
    # Create the directory if it doesn't exist
    os.makedirs(directory, exist_ok=True)
    
    # Join the directory path with the file name
    results_path = os.path.join(directory, name)
    
    # Check if the file exists
    file_exists = os.path.isfile(results_path)
    
    # Open the file in append mode or write mode depending on whether the file exists
    with open(results_path, 'a' if file_exists else 'w') as f:
        # Redirect stdout to the file
        sys.stdout = f
        print(" ---------------------------------------------------------------------------------------------------------------------- ")
        print(" ----------------------------- Model parameters ------------------------------")
        print("model name:" + modelName)
        print('learning rate  = 0.001')
        print('epochs = 50')
        print('folds = 5')
        print('batch size = 64')
        print('Test set size  = ',test_set_size)
        print("enlarge_factor =", enlarge_factor)
        print('loss = '+ loss)
        print('optimizer = '+ optim)
        print(" ------------------------------- Evaluation  ---------------------------------")
        evaluate_network(model, test_loader)
        print(" ---------------------------------------------------------------------------------------------------------------------- ")
 
    # Reset stdout to its default value (console)
    sys.stdout = sys.__stdout__


In [13]:
# DEFINE THE FIT FUNCTION

def fit(train_loader, val_loader, model, criterion, optimizer, n_epochs, l2 ,to_device=True):
    #Train the network
    loss_values = []
    balanced_accuracy_values = []
    l2_lambda=l2
    for epoch in range(n_epochs):

        model.train()
        running_loss = 0.0

        for X, y in train_loader:
            # Move data to GPU
            X, y = X.to(device), y.to(device)

            #Forward pass
            y_pred = model(X)

            #if criterion -> MSELoss
            #y_one_hot = F.one_hot(y, num_classes=9).float()
            if isinstance(criterion, nn.MSELoss):
                y_one_hot = F.one_hot(y, num_classes=9).float()
                loss = criterion(y_pred, y_one_hot)
            else:
                loss = criterion(y_pred, y)


            # L2 regularization
            l2_reg = 0
            for param in model.parameters():
                l2_reg += torch.norm(param)**2

            loss += l2_lambda * l2_reg

            running_loss += loss.item()

            #Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_loss = running_loss / len(train_loader)
        train_acc = balanced_accuracy_score(y.tolist(),y_pred.argmax(dim=1).tolist())

        #Validation
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
          for X, y in val_loader:
            X, y = X.to(device), y.to(device)

            y_pred = model(X)
            if isinstance(criterion, nn.MSELoss):
                y_one_hot = F.one_hot(y, num_classes=9).float()
                loss = criterion(y_pred, y_one_hot)
            else:
                loss = criterion(y_pred, y)
            val_loss += loss.item()

        val_loss = val_loss/len(val_loader)
        val_acc = balanced_accuracy_score(y.tolist(),y_pred.argmax(dim=1).tolist())

        print(f'Epoch [{epoch + 1}/{n_epochs}], '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%')

    return train_loss, balanced_accuracy_values, model

In [14]:
def Train_K_FOLDS(k_model, epochs, lr, folds, batch_size, l2, input_size, hidden_size, output_size, optim_type, loss_type):

    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

    models = []
    index=0
    i = 0
    best_accuracy = 0
    best_model = k_model(input_size, hidden_size, output_size)
    second_best_model = k_model(input_size, hidden_size, output_size)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for fold, (train_index, val_index) in enumerate(kf.split(train_val_dataset)):

        model = k_model(input_size, hidden_size, output_size)
        model.to(device)
        second_best_model.to(device)

        #Creating DataLoaders for training and validation
        train_sampler = torch.utils.data.SubsetRandomSampler(train_index)
        val_sampler = torch.utils.data.SubsetRandomSampler(val_index)
        #
        train_loader = DataLoader(train_val_dataset, batch_size=batch_size, sampler=train_sampler)
        val_loader = DataLoader(train_val_dataset, batch_size=batch_size, sampler=val_sampler)

        #CHOOSE LOSS
        if loss_type == 'MSELoss' or loss_type == 'BCELoss':
            criterion = nn.MSELoss()
            
        elif loss_type == 'cross_entropy':
            criterion = nn.CrossEntropyLoss()
        elif loss_type == 'BCELoss':
            criterion = nn.BCELoss()
        else:
            raise ValueError(f"Unsupported loss type: {loss_type}")
            
        #CHOOSE OPTIMIZER
        if optim_type == 'adam':
            optimizer = optim.Adam(model.parameters(), lr=lr)
        elif optim_type == 'SGD':
            optimizer = optim.SGD(model.parameters(), lr=lr)
        elif optim_type == 'RMSprop':
            optimizer = optim.RMSprop(model.parameters(), lr=lr)
        else:
            raise ValueError(f"Unsupported optimizer type: {optim_type}")

        # Train the model
        train_loss, train_acc,  trained_model = fit(train_loader, val_loader, model, criterion, optimizer, epochs, l2, to_device=False)
        #models.append(trained_model)
        conf_matrix, F_score, bal_acc = evaluate_network(trained_model, val_loader)
        if(bal_acc > best_accuracy):
            best_accuracy = bal_acc
            index = i
            if(fold >1):second_best_model = best_model
            best_model = model

        print("Best model has index ", index , "\n")


    models.append(best_model)
    models.append(second_best_model)

    return best_model, models

#MLP


In [15]:
class MLP_v1(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.1):
        super(MLP_v1, self).__init__()
        self.lin1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()

        self.dropout = Dropout(dropout_rate)
        #hidden
        self.lin2 = nn.Linear(hidden_size, hidden_size)

        self.lin3 = nn.Linear(hidden_size, hidden_size)

        #hidden
        self.lin4 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input tensor
        x = self.lin1(x)
        x = self.relu(x)
        x = self.dropout(x)  # Apply dropout
        #hidden
        x = self.lin2(x)
        x = self.relu(x)
        x = self.dropout(x)  # Apply dropout

        x = self.lin3(x)
        x = self.relu(x)
        x = self.dropout(x)  # Apply dropout

        #hidden
        x = self.lin4(x)
        return x
    
    def get_architecture_name(self):
        name = 'MLP_v1'
        return name

In [16]:
class MLP_v2(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP_v2, self).__init__()
        self.lin1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()

        #hidden
        self.lin2 = nn.Linear(hidden_size, hidden_size)

        self.lin3 = nn.Linear(hidden_size, hidden_size)

        #hidden
        self.lin4 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input tensor
        x = self.lin1(x)
        x = self.relu(x)

        #hidden
        x = self.lin2(x)
        x = self.relu(x)

        x = self.lin3(x)
        x = self.relu(x)

        #hidden
        x = self.lin4(x)
        return x
    
    def get_architecture_name(self):
        name = 'MLP_v2'
        return name

In [17]:
class MLP_v3(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.1):
        super(MLP_v3, self).__init__()
        self.lin1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()

        self.dropout = Dropout(dropout_rate)
        #hidden
        self.lin2 = nn.Linear(hidden_size, hidden_size)

        self.lin3 = nn.Linear(hidden_size, hidden_size)

        self.lin4 = nn.Linear(hidden_size, hidden_size)

        self.lin5 = nn.Linear(hidden_size, hidden_size)

        #hidden
        self.lin6 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input tensor
        x = self.lin1(x)
        x = self.relu(x)
        x = self.dropout(x)  # Apply dropout
        #hidden
        x = self.lin2(x)
        x = self.relu(x)
        x = self.dropout(x)  # Apply dropout

        x = self.lin3(x)
        x = self.relu(x)
        x = self.dropout(x)  # Apply dropout

        x = self.lin4(x)
        x = self.relu(x)
        x = self.dropout(x)  # Apply dropout

        x = self.lin4(x)
        x = self.relu(x)
        x = self.dropout(x)  # Apply dropout
        #hidden
        x = self.lin6(x)
        return x
    
    def get_architecture_name(self):
        name = 'MLP_v3'
        return name

In [18]:
class MLP_v4(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP_v4, self).__init__()
        self.lin1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()

        #hidden
        self.lin2 = nn.Linear(hidden_size, hidden_size)

        self.lin3 = nn.Linear(hidden_size, hidden_size)

        self.lin4 = nn.Linear(hidden_size, hidden_size)

        self.lin5 = nn.Linear(hidden_size, hidden_size)

        #hidden
        self.lin6 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input tensor
        x = self.lin1(x)
        x = self.relu(x)

        #hidden
        x = self.lin2(x)
        x = self.relu(x)

        x = self.lin3(x)
        x = self.relu(x)

        x = self.lin4(x)
        x = self.relu(x)

        x = self.lin4(x)
        x = self.relu(x)

        #hidden
        x = self.lin6(x)
        return x
    
    def get_architecture_name(self):
        name = 'MLP_v4'
        return name

In [19]:
class MLP_v4_sig(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP_v4_sig, self).__init__()
        self.lin1 = nn.Linear(input_size, hidden_size)
        self.sigmoid = nn.Sigmoid()  # Change ReLU to Sigmoid

        # Hidden
        self.lin2 = nn.Linear(hidden_size, hidden_size)

        self.lin3 = nn.Linear(hidden_size, hidden_size)

        self.lin4 = nn.Linear(hidden_size, hidden_size)

        self.lin5 = nn.Linear(hidden_size, hidden_size)

        # Hidden
        self.lin6 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input tensor
        x = self.lin1(x)
        x = self.sigmoid(x)  # Change ReLU to Sigmoid

        # Hidden
        x = self.lin2(x)
        x = self.sigmoid(x)  # Change ReLU to Sigmoid

        x = self.lin3(x)
        x = self.sigmoid(x)  # Change ReLU to Sigmoid

        x = self.lin4(x)
        x = self.sigmoid(x)  # Change ReLU to Sigmoid

        x = self.lin4(x)
        x = self.sigmoid(x)  # Change ReLU to Sigmoid

        # Hidden
        x = self.lin6(x)
        return x
    
    def get_architecture_name(self):
        name = 'MLP_v4_sig'
        return name


In [20]:
class MLP_v4_tanh(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP_v4_tanh, self).__init__()
        self.lin1 = nn.Linear(input_size, hidden_size)
        self.tanh = nn.Tanh()  # Change ReLU to Tanh

        # Hidden
        self.lin2 = nn.Linear(hidden_size, hidden_size)

        self.lin3 = nn.Linear(hidden_size, hidden_size)

        self.lin4 = nn.Linear(hidden_size, hidden_size)

        self.lin5 = nn.Linear(hidden_size, hidden_size)

        # Hidden
        self.lin6 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input tensor
        x = self.lin1(x)
        x = self.tanh(x)  # Change ReLU to Tanh

        # Hidden
        x = self.lin2(x)
        x = self.tanh(x)  # Change ReLU to Tanh

        x = self.lin3(x)
        x = self.tanh(x)  # Change ReLU to Tanh

        x = self.lin4(x)
        x = self.tanh(x)  # Change ReLU to Tanh

        x = self.lin4(x)
        x = self.tanh(x)  # Change ReLU to Tanh

        # Hidden
        x = self.lin6(x)
        return x
    
    def get_architecture_name(self):
        name = 'MLP_v4_tanh'
        return name


In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

#Initialize model, loss function, and optimizer
input_size = IMG_WIDTH * IMG_HEIGHT * 3 #RGB
#n of classes
output_size = 9

save_dir = '/home/stefanotrenti/AML/project/AOL_test'
name = 'AOL_function_tests_MLP.txt'

optimizers = ['adam','SGD','RMSprop']
losses = ['cross_entropy','MSELoss','BCELoss']


best_model_v4, _ = Train_K_FOLDS(MLP_v4, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'adam', 'cross_entropy')
filename_v4 = best_model_v4.get_architecture_name() + '.txt'
saveResults(best_model_v4,save_dir,name,filename_v4, 'adam', 'cross_entropy')

best_model_v4, _ = Train_K_FOLDS(MLP_v4, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'adam', 'MSELoss')
filename_v4 = best_model_v4.get_architecture_name() + '.txt'
saveResults(best_model_v4,save_dir,name,filename_v4, 'adam', 'MSELoss')

best_model_v4, _ = Train_K_FOLDS(MLP_v4, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'adam', 'BCELoss')
filename_v4 = best_model_v4.get_architecture_name() + '.txt'
saveResults(best_model_v4,save_dir,name,filename_v4, 'adam', 'BCELoss')


best_model_v4, _ = Train_K_FOLDS(MLP_v4, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'SGD', 'cross_entropy')
filename_v4 = best_model_v4.get_architecture_name() + '.txt'
saveResults(best_model_v4,save_dir,name,filename_v4, 'SGD', 'cross_entropy')

best_model_v4, _ = Train_K_FOLDS(MLP_v4, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'SGD', 'MSELoss')
filename_v4 = best_model_v4.get_architecture_name() + '.txt'
saveResults(best_model_v4,save_dir,name,filename_v4, 'SGD', 'MSELoss')

best_model_v4, _ = Train_K_FOLDS(MLP_v4, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'SGD', 'BCELoss')
filename_v4 = best_model_v4.get_architecture_name() + '.txt'
saveResults(best_model_v4,save_dir,name,filename_v4, 'SGD', 'BCELoss')


best_model_v4, _ = Train_K_FOLDS(MLP_v4, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'RMSprop', 'cross_entropy')
filename_v4 = best_model_v4.get_architecture_name() + '.txt'
saveResults(best_model_v4,save_dir,name,filename_v4, 'RMSprop', 'cross_entropy')

best_model_v4, _ = Train_K_FOLDS(MLP_v4, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'RMSprop', 'MSELoss')
filename_v4 = best_model_v4.get_architecture_name() + '.txt'
saveResults(best_model_v4,save_dir,name,filename_v4, 'RMSprop', 'MSELoss')

best_model_v4, _ = Train_K_FOLDS(MLP_v4, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'RMSprop', 'BCELoss')
filename_v4 = best_model_v4.get_architecture_name() + '.txt'
saveResults(best_model_v4,save_dir,name,filename_v4, 'RMSprop', 'BCELoss')

cuda:0
Epoch [1/50], Train Loss: 4.2380, Train Acc: 0.00%, Val Loss: 2.1700, Val Acc: 11.11%
Epoch [2/50], Train Loss: 3.9778, Train Acc: 16.67%, Val Loss: 2.1666, Val Acc: 11.11%
Epoch [3/50], Train Loss: 3.7938, Train Acc: 12.50%, Val Loss: 2.1502, Val Acc: 12.96%
Epoch [4/50], Train Loss: 3.6401, Train Acc: 26.67%, Val Loss: 2.1055, Val Acc: 16.67%
Epoch [5/50], Train Loss: 3.5014, Train Acc: 13.33%, Val Loss: 2.0372, Val Acc: 22.50%
Epoch [6/50], Train Loss: 3.3576, Train Acc: 20.00%, Val Loss: 1.9458, Val Acc: 21.85%
Epoch [7/50], Train Loss: 3.2340, Train Acc: 37.50%, Val Loss: 1.9064, Val Acc: 32.73%
Epoch [8/50], Train Loss: 3.1430, Train Acc: 28.57%, Val Loss: 1.8038, Val Acc: 32.50%
Epoch [9/50], Train Loss: 3.0579, Train Acc: 22.22%, Val Loss: 1.6871, Val Acc: 38.75%
Epoch [10/50], Train Loss: 2.8942, Train Acc: 35.71%, Val Loss: 1.5365, Val Acc: 31.11%
Epoch [11/50], Train Loss: 2.7445, Train Acc: 58.33%, Val Loss: 1.4232, Val Acc: 32.59%
Epoch [12/50], Train Loss: 2.6139, 

In [22]:
best_model_v4_sig, _ = Train_K_FOLDS(MLP_v4_sig, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'adam', 'cross_entropy')
filename_v4_sig = best_model_v4_sig.get_architecture_name() + '.txt'
saveResults(best_model_v4_sig,save_dir,name,filename_v4_sig, 'adam', 'cross_entropy')

best_model_v4_sig, _ = Train_K_FOLDS(MLP_v4_sig, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'adam', 'MSELoss')
filename_v4_sig = best_model_v4_sig.get_architecture_name() + '.txt'
saveResults(best_model_v4_sig,save_dir,name,filename_v4_sig, 'adam', 'MSELoss')

best_model_v4_sig, _ = Train_K_FOLDS(MLP_v4_sig, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'adam', 'BCELoss')
filename_v4_sig = best_model_v4_sig.get_architecture_name() + '.txt'
saveResults(best_model_v4_sig,save_dir,name,filename_v4_sig, 'adam', 'BCELoss')


best_model_v4_sig, _ = Train_K_FOLDS(MLP_v4_sig, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'SGD', 'cross_entropy')
filename_v4_sig = best_model_v4_sig.get_architecture_name() + '.txt'
saveResults(best_model_v4_sig,save_dir,name,filename_v4_sig, 'SGD', 'cross_entropy')

best_model_v4_sig, _ = Train_K_FOLDS(MLP_v4_sig, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'SGD', 'MSELoss')
filename_v4_sig = best_model_v4_sig.get_architecture_name() + '.txt'
saveResults(best_model_v4_sig,save_dir,name,filename_v4_sig, 'SGD', 'MSELoss')

best_model_v4_sig, _ = Train_K_FOLDS(MLP_v4_sig, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'SGD', 'BCELoss')
filename_v4_sig = best_model_v4_sig.get_architecture_name() + '.txt'
saveResults(best_model_v4_sig,save_dir,name,filename_v4_sig, 'SGD', 'BCELoss')


best_model_v4_sig, _ = Train_K_FOLDS(MLP_v4_sig, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'RMSprop', 'cross_entropy')
filename_v4_sig = best_model_v4_sig.get_architecture_name() + '.txt'
saveResults(best_model_v4_sig,save_dir,name,filename_v4_sig, 'RMSprop', 'cross_entropy')

best_model_v4_sig, _ = Train_K_FOLDS(MLP_v4_sig, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'RMSprop', 'MSELoss')
filename_v4_sig = best_model_v4_sig.get_architecture_name() + '.txt'
saveResults(best_model_v4_sig,save_dir,name,filename_v4_sig, 'RMSprop', 'MSELoss')

best_model_v4_sig, _ = Train_K_FOLDS(MLP_v4_sig, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'RMSprop', 'BCELoss')
filename_v4_sig = best_model_v4_sig.get_architecture_name() + '.txt'
saveResults(best_model_v4_sig,save_dir,name,filename_v4_sig, 'RMSprop', 'BCELoss')




best_model_v4_tanh, _ = Train_K_FOLDS(MLP_v4_tanh, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'adam', 'cross_entropy')
filename_v4_tanh = best_model_v4_tanh.get_architecture_name() + '.txt'
saveResults(best_model_v4_tanh,save_dir,name,filename_v4_tanh, 'adam', 'cross_entropy')

best_model_v4_tanh, _ = Train_K_FOLDS(MLP_v4_tanh, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'adam', 'MSELoss')
filename_v4_tanh = best_model_v4_tanh.get_architecture_name() + '.txt'
saveResults(best_model_v4_tanh,save_dir,name,filename_v4_tanh, 'adam', 'MSELoss')

best_model_v4_tanh, _ = Train_K_FOLDS(MLP_v4_tanh, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'adam', 'BCELoss')
filename_v4_tanh = best_model_v4_tanh.get_architecture_name() + '.txt'
saveResults(best_model_v4_tanh,save_dir,name,filename_v4_tanh, 'adam', 'BCELoss')


best_model_v4_tanh, _ = Train_K_FOLDS(MLP_v4_tanh, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'SGD', 'cross_entropy')
filename_v4_tanh = best_model_v4_tanh.get_architecture_name() + '.txt'
saveResults(best_model_v4_tanh,save_dir,name,filename_v4_tanh, 'SGD', 'cross_entropy')

best_model_v4_tanh, _ = Train_K_FOLDS(MLP_v4_tanh, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'SGD', 'MSELoss')
filename_v4_tanh = best_model_v4_tanh.get_architecture_name() + '.txt'
saveResults(best_model_v4_tanh,save_dir,name,filename_v4_tanh, 'SGD', 'MSELoss')

best_model_v4_tanh, _ = Train_K_FOLDS(MLP_v4_tanh, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'SGD', 'BCELoss')
filename_v4_tanh = best_model_v4_tanh.get_architecture_name() + '.txt'
saveResults(best_model_v4_tanh,save_dir,name,filename_v4_tanh, 'SGD', 'BCELoss')


best_model_v4_tanh, _ = Train_K_FOLDS(MLP_v4_tanh, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'RMSprop', 'cross_entropy')
filename_v4_tanh = best_model_v4_tanh.get_architecture_name() + '.txt'
saveResults(best_model_v4_tanh,save_dir,name,filename_v4_tanh, 'RMSprop', 'cross_entropy')

best_model_v4_tanh, _ = Train_K_FOLDS(MLP_v4_tanh, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'RMSprop', 'MSELoss')
filename_v4_tanh = best_model_v4_tanh.get_architecture_name() + '.txt'
saveResults(best_model_v4_tanh,save_dir,name,filename_v4_tanh, 'RMSprop', 'MSELoss')

best_model_v4_tanh, _ = Train_K_FOLDS(MLP_v4_tanh, 50, 0.0001, 10, 32, 0.01, input_size, 128, output_size, 'RMSprop', 'BCELoss')
filename_v4_tanh = best_model_v4_tanh.get_architecture_name() + '.txt'
saveResults(best_model_v4_tanh,save_dir,name,filename_v4_tanh, 'RMSprop', 'BCELoss')

Epoch [1/50], Train Loss: 4.3325, Train Acc: 16.67%, Val Loss: 2.2999, Val Acc: 12.50%
Epoch [2/50], Train Loss: 4.0017, Train Acc: 16.67%, Val Loss: 2.2332, Val Acc: 14.29%
Epoch [3/50], Train Loss: 3.7864, Train Acc: 14.29%, Val Loss: 2.1869, Val Acc: 11.11%
Epoch [4/50], Train Loss: 3.6361, Train Acc: 16.67%, Val Loss: 2.1496, Val Acc: 12.50%
Epoch [5/50], Train Loss: 3.5252, Train Acc: 12.50%, Val Loss: 2.1227, Val Acc: 12.50%
Epoch [6/50], Train Loss: 3.4285, Train Acc: 16.67%, Val Loss: 2.1049, Val Acc: 14.29%
Epoch [7/50], Train Loss: 3.3502, Train Acc: 16.67%, Val Loss: 2.0930, Val Acc: 11.11%
Epoch [8/50], Train Loss: 3.2759, Train Acc: 0.00%, Val Loss: 2.0857, Val Acc: 11.11%
Epoch [9/50], Train Loss: 3.2055, Train Acc: 12.50%, Val Loss: 2.0799, Val Acc: 12.50%
Epoch [10/50], Train Loss: 3.1433, Train Acc: 12.50%, Val Loss: 2.0784, Val Acc: 12.50%
Epoch [11/50], Train Loss: 3.0866, Train Acc: 16.67%, Val Loss: 2.0761, Val Acc: 11.11%
Epoch [12/50], Train Loss: 3.0320, Train A

#Training


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

#Initialize model, loss function, and optimizer
input_size = IMG_WIDTH * IMG_HEIGHT * 3 #RGB
#n of classes
output_size = 9

batchSizes = [32, 64]
hiddens = [64, 128]
folds = [10]
lrs = [0.001, 0.0001]
l2s = [0.01, 0.001] #l2 regulrization
time = [50, 100]
#save_dir = '/content/drive/MyDrive/Nicoli'
save_dir = '/home/stefanotrenti/AML/project/MLP/'
#save_dir = 'MLP/'

for b in range(len(batchSizes)):
  batch_size = batchSizes[b]
  for f in range(len(folds)):
    fold = folds[f]
    for h in range(len(hiddens)):
      hidden_size = hiddens[h]
      for l in range(len(lrs)):
        lr = lrs[l]
        for li in range(len(l2s)):
          l2 = l2s[li]
          for e in range(len(time)):
            epochs = time[e]
            best_model_v1, _ = Train_K_FOLDS(MLP_v1, epochs, lr, fold, batch_size, l2, input_size, hidden_size, output_size)
            filename_v1 = best_model_v1.get_architecture_name() + '.txt'
            saveResults(best_model_v1, save_dir, filename_v1, modelName)
            best_model_v2, _ = Train_K_FOLDS(MLP_v2, epochs, lr, fold, batch_size, l2, input_size, hidden_size, output_size)
            filename_v2 = best_model_v2.get_architecture_name() + '.txt'
            saveResults(best_model_v2, save_dir, filename_v2, modelName)
            best_model_v3, _ = Train_K_FOLDS(MLP_v3, epochs, lr, fold, batch_size, l2, input_size, hidden_size, output_size)
            filename_v3 = best_model_v3.get_architecture_name() + '.txt'
            saveResults(best_model_v3, save_dir, filename_v3, modelName)
            best_model_v4, _ = Train_K_FOLDS(MLP_v4, epochs, lr, fold, batch_size, l2, input_size, hidden_size, output_size)
            modelName = "_"+str(batch_size)+"_"+str(fold)+"_"+str(lr)+"_"+str(epochs)+"_"+str(hidden_size)+"_"+str(l2)
            filename_v4 = best_model_v4.get_architecture_name() + '.txt'
            saveResults(best_model_v4, save_dir, filename_v4, modelName)
"""
best_model, _ = Train_K_FOLDS(MLP_v1, epochs, lr, fold, batch_size, l2, input_size, hidden_size, output_size)
best_model, _ = Train_K_FOLDS(MLP_v2, epochs, lr, fold, batch_size, l2, input_size, hidden_size, output_size)
best_model, _ = Train_K_FOLDS(MLP_v3, epochs, lr, fold, batch_size, l2, input_size, hidden_size, output_size)
best_model, _ = Train_K_FOLDS(MLP_v4, epochs, lr, fold, batch_size, l2, input_size, hidden_size, output_size)
"""